In [ ]:
#--- Imports ---#

import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.losses import MeanSquaredError,mean_squared_logarithmic_error
from sklearn.model_selection import train_test_split
# calculation and plot
import numpy as np
import numpy.linalg as LA
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
%matplotlib inline
# data processing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# predictor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

from keras.utils import plot_model
from IPython.display import Image

In [ ]:
np.random.seed(42)  # For reproducibility

# Loading Composition Data

In [ ]:
#--- Print input directory ---#

print("Input Directory:")

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#--- Load composition data ---#

# load train composition data
zip_comp_train = '/kaggle/input/nomad2018-predict-transparent-conductors/train.csv.zip' # define path to `.zip` folder for train data
csv_comp_train = ZipFile(zip_comp_train).open("train.csv")                              # open `.zip` folder to get `.csv` file as a `zipfile` object
df_comp_train  = pd.read_csv(csv_comp_train, encoding='unicode_escape')                 # read the `.csv` file into a dataframe
size_train     = len(df_comp_train)                                                     # number of samples in train data
df_comp_train['dataset'] = 'train'                                                      # create new column to indicate data is from train data

# load test composition data
zip_comp_test = '/kaggle/input/nomad2018-predict-transparent-conductors/test.csv.zip'   # define path to `.zip` folder for test data
csv_comp_test = ZipFile(zip_comp_test).open("test.csv")                                 # open `.zip` folder to get `.csv` file as a `zipfile` object
df_comp_test  = pd.read_csv(csv_comp_test, encoding='unicode_escape')                   # read the `.csv` file into a dataframe
size_test     = len(df_comp_test)                                                       # number of samples in test data
df_comp_test['dataset'] = 'test'                                                        # create new column to indicate data is from test data

# merge train and test set
df_comp  = pd.concat([df_comp_train, df_comp_test], axis=0, ignore_index=True, sort=False)
size_all = len(df_comp)

#--- Rename Columns ---#
df_comp.rename(columns={
    'number_of_total_atoms' : 'numAtoms',
    'percent_atom_al' : 'xAl',
    'percent_atom_ga' : 'xGa',
    'percent_atom_in' : 'xIn',
    'lattice_vector_1_ang' : 'a',
    'lattice_vector_2_ang' : 'b',
    'lattice_vector_3_ang' : 'c',
    'lattice_angle_alpha_degree' : 'alpha',
    'lattice_angle_beta_degree' : 'beta',
    'lattice_angle_gamma_degree' : 'gamma',
    'formation_energy_ev_natom' : 'formationE',
    'bandgap_energy_ev' : 'bandgapE'}, inplace=True)

# find features and targets
targets = ['formationE', 'bandgapE']
features = [column for column in df_comp.columns if column not in targets]
features.remove('id')
features.remove('dataset')

# print composition data
print(f"Size of train data: {size_train}")
print(f"Size of test data: {size_test}")
print(f"Size of all data: {size_all}")
print(f"\nFeature columns: \n{features}")
print(f"\nTarget columns: \n{targets}")
df_comp

# Loading Coordinate Data

In [ ]:
#--- Function for reading .xyz files ---#

def read_xyz_file(zip_folder, xyz_file_name):
    
    data                  = [] # a list to hold individual data entries read from the file
    list_coordinates      = [] # a list of atom 3-dimensional coordinates, e.g. (x, y, z)
    list_elements         = [] # a list of atom elements
    list_lattice_vectors  = [] # a list of arrays of lattice data [(a1, b1, c1), (a2, b2, c2), (a3, b3, c3)]
    
    with zip_folder.open(xyz_file_name) as file:
        
        for line in file.readlines():
            
            data = line.decode().split()
            
            if data[0] == 'atom':
                list_coordinates.append(np.array(data[1:4], dtype=np.float64))
                list_elements.append(data[4])
                
            elif data[0] == 'lattice_vector':
                list_lattice_vectors.append(np.array(data[1:4], dtype=np.float64))
    
    return list_coordinates, list_elements, list_lattice_vectors

In [ ]:
#--- Function to convert element to charge ---#

dict_element_charge = {'O': 8, 'Al': 13, 'Ga': 31, 'In': 49}

def element_to_charge(list_elements, dict_element_charge):
    
    list_charges = np.transpose(np.array([[dict_element_charge[element] for element in list_elements]]))
    
    return list_charges

In [ ]:
#--- Function for producing sine matrix ---# 

def create_sine_matrix(list_coordinates, list_charges, list_lattice_vectors, matrix_dim, debug=False):
    
    num_atom = len(list_coordinates) # number of atoms
    
    distance_matrix = np.ones((matrix_dim, matrix_dim))
    charge_matrix = np.zeros((matrix_dim, matrix_dim)) 
    
    A = np.transpose(list_lattice_vectors) # A = [(a1, a2, a3), (b1, b2, b3), (c1, c2, c3)]
    B = LA.inv(A)                          # inverse matrix of A
    
    # populate distance matrix
    for i in range(num_atom):
        for j in range(i):
            r_ij = np.dot(B, list_coordinates[i] - list_coordinates[j])
            sin_sq_r = (np.sin(np.pi * r_ij))**2
            distance = LA.norm(np.dot(A, sin_sq_r))
            distance_matrix[i, j], distance_matrix[j, i] = distance, distance
    
    # create charge matrix
    matrix  = np.dot(list_charges, np.transpose(list_charges)).astype(np.float64)
    matrix -= np.diag(np.diag(matrix))                                            # let diagonal components be zero
    matrix += np.diag(0.5 * list_charges**2.4).astype(np.float64)                 # from definition
    
    charge_matrix[:len(matrix), :len(matrix)] += matrix
    
    # create sine matrix
    sine_matrix = charge_matrix / distance_matrix
    
    if debug == True:
        return sine_matrix, distance_matrix, charge_matrix
    else:
        return sine_matrix

In [ ]:
#--- Function for producing eigen spectrum ---#

def create_eigenspectrum(sine_matrix, matrix_dim):
    
    spectrum = np.full((len(sine_matrix)), 0)
    
    # Identify the rows that are not entirely zero (i.e., non-padded rows)
    indices_nopad = np.any(sine_matrix != 0, axis=1)
    
    # Extract the submatrix that does not include padding
    matrix_nopad = sine_matrix[indices_nopad][:, indices_nopad]
    
    # Compute the eigenspectrum on the non-padded submatrix
    spectrum_nopad = LA.eigvalsh(matrix_nopad)
    spectrum_nopad = np.sort(spectrum_nopad)[::-1]
    
    spectrum[indices_nopad] = spectrum_nopad
    
    return spectrum

In [ ]:
#--- Process coordinate data ---#

list_spectrum = []
list_sine_matrix = []

# Coordinates in .xyz files.
files_train = ZipFile('/kaggle/input/nomad2018-predict-transparent-conductors/train.zip')
files_test = ZipFile('/kaggle/input/nomad2018-predict-transparent-conductors/test.zip')

for index in range(size_all):
    if index%500 == 0:
        print(index)
    
    dataset_label = df_comp.dataset.values[index]
    row_id = df_comp.id.values[index]
    filename = "{}/{}/geometry.xyz".format(dataset_label, row_id)
    files = files_train
    if dataset_label == "test":
        files = files_test
    # file processing
    list_coordinates, list_elements, list_lattice_vectors = read_xyz_file(files, filename)
    list_charges = element_to_charge(list_elements, dict_element_charge)
    sine_matrix = create_sine_matrix(list_coordinates, list_charges, list_lattice_vectors, matrix_dim=80)
    list_sine_matrix.append(sine_matrix)
    # eigen spectrum
    spectrum = create_eigenspectrum(sine_matrix, matrix_dim=80)
    
    list_spectrum.append(spectrum)

# Data Preprocessing

In [ ]:
df_spectrum = pd.DataFrame(list_spectrum).astype(np.float64)
df_spectrum = df_spectrum.fillna(0)

In [ ]:
# standard scaling
ss = StandardScaler()
df_spectrum_std = pd.DataFrame(ss.fit_transform(df_spectrum.values))
# PCA
pca = PCA(n_components=80)
df_spectrum_pca = pd.DataFrame(pca.fit_transform(df_spectrum_std.values))

df_spectrum_pca.head()

In [ ]:
df_comp_final = df_comp.drop(['dataset'], axis=1)
# df.drop(['id', 'spacegroup', 'number_of_total_atoms'], axis=1, inplace=True)
df_comp_final = df_comp_final.drop(['id'], axis=1)

In [ ]:
indices_dropped = [395, 126, 1215, 1886, 2075, 353, 308, 2154, 531, 1379, 2319, 2337, 2370, 2333]
df_comp_final.drop(indices_dropped, axis=0, inplace=True)
df_spectrum_final = df_spectrum_pca.drop(indices_dropped, axis=0)
list_sine_matrix_final = [matrix for idx, matrix in enumerate(list_sine_matrix) if idx not in indices_dropped]

# Data Splitting

In [ ]:
#--- Get sizes of test, train, and validation datasets ---#

# get size of final combined (test and train) data and size of non-test data
size_final   = len(df_comp_final)
size_nontest = size_final - size_test

# set fraction of non-test data used for validation and get size of validation data
frac_val = 0.2                        
size_val = int(size_nontest * frac_val)

# get indices for validation and training data
indices_val   = np.random.choice(size_nontest, size=size_val, replace=False) 
indices_train = np.setdiff1d(np.arange(size_nontest), indices_val)

#--- Split data between test, train, and validation datasets ---#

# split composition data
df_comp_test   = df_comp_final.drop(['formationE', 'bandgapE'], axis=1)[size_nontest:]
df_comp_nontest = df_comp_final.drop(['formationE', 'bandgapE'], axis=1)[:size_nontest]
df_comp_train  = df_comp_nontest.iloc[indices_train]
df_comp_val    = df_comp_nontest.iloc[indices_val]

# split sine matrix data
list_sine_matrix_test   = list_sine_matrix_final[size_nontest:]
list_sine_matrix_nontest = list_sine_matrix_final[:size_nontest]
list_sine_matrix_train  = [list_sine_matrix_nontest[idx] for idx in indices_train]
list_sine_matrix_val    = [list_sine_matrix_nontest[idx] for idx in indices_val]

# split spectrum data
df_spectrum_test   = df_spectrum_final[size_nontest:]
df_spectrum_nontest = df_spectrum_final[:size_nontest]
df_spectrum_train  = df_spectrum_nontest.iloc[indices_train]
df_spectrum_val    = df_spectrum_nontest.iloc[indices_val]

# reshape spectrum data for lstm
arr_spectrum_test  = np.array(df_spectrum_test).reshape((len(df_spectrum_test),80,1))
arr_spectrum_train = np.array(df_spectrum_train).reshape((len(df_spectrum_train),80,1))
arr_spectrum_val   = np.array(df_spectrum_val).reshape((len(df_spectrum_val),80,1))

# get target data
y_formation = df_comp_final['formationE'][:size_nontest].values
y_bandgap   = df_comp_final['bandgapE'][:size_nontest].values

# split formation energy target data
y_formation_train = y_formation[indices_train]
y_formation_val   = y_formation[indices_val]

# split bandgap energy target data
y_bandgap_train = y_bandgap[indices_train]
y_bandgap_val   = y_bandgap[indices_val]

# combine target data
y_train = np.column_stack((y_formation_train, y_bandgap_train))
y_val   = np.column_stack((y_formation_val, y_bandgap_val))

# Models

In [ ]:
#--- Imports ---#

from keras.layers import Dense, BatchNormalization, Dropout, Bidirectional, LSTM, Input, GRU
from keras.losses import MeanSquaredError,mean_squared_logarithmic_error
from keras.callbacks import Callback, LearningRateScheduler, EarlyStopping
from keras.regularizers import l1,l2
import keras.backend as K
from keras.models import Model
from keras.layers import concatenate

#--- Configuration ---#

batch_size = 128
drop_out = 0.2

#--- Loss Functions ---#

def rmsle(y_true, y_pred):
    # Clip values to avoid negative values in logarithm
    y_true_clipped = K.clip(y_true, K.epsilon(), None)
    y_pred_clipped = K.clip(y_pred, K.epsilon(), None)
    
    # Calculate RMSLE
    log_diff = K.log(y_true_clipped + 1) - K.log(y_pred_clipped + 1)
    return K.sqrt(K.mean(K.square(log_diff), axis=-1))

msle = mean_squared_logarithmic_error

#--- Callbacks ---#

cb_earlystop = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

callbacks = [cb_earlystop]

# Composite Model

In [ ]:
input_shape_ff   = (len(df_comp_train.iloc[0]),)
input_shape_lstm = (80,1)

X_train = df_comp_train, arr_spectrum_train, arr_spectrum_train
X_val = df_comp_val, arr_spectrum_val, arr_spectrum_val
X_test = df_comp_test, arr_spectrum_test, arr_spectrum_test

In [ ]:
def create_model(num_targets=1):

    ff_input     = Input(shape=input_shape_ff)
    ff_dense_1   = Dense(1024, activation='relu')(ff_input)
    ff_norm_1    = BatchNormalization()(ff_dense_1)
    ff_drop_1    = Dropout(drop_out)(ff_norm_1)
    ff_dense_2   = Dense(512, activation='relu')(ff_drop_1)
    ff_norm_2    = BatchNormalization()(ff_dense_2)
    ff_drop_2    = Dropout(drop_out)(ff_norm_2)
    ff_dense_3   = Dense(258, activation='relu')(ff_drop_2)
    ff_norm_3    = BatchNormalization()(ff_dense_3)
    ff_drop_3    = Dropout(drop_out)(ff_norm_3)
    ff_dense_4   = Dense(128, activation='relu')(ff_drop_3)
    ff_norm_4    = BatchNormalization()(ff_dense_4)
    ff_drop_4    = Dropout(drop_out)(ff_norm_4)
    ff_dense_5   = Dense(64, activation='relu')(ff_drop_4)
    ff_norm_5    = BatchNormalization()(ff_dense_5)
    ff_drop_5    = Dropout(drop_out)(ff_norm_5)
    ff_dense_6   = Dense(32, activation='relu')(ff_drop_5)
    ff_norm_6    = BatchNormalization()(ff_dense_6)
    ff_drop_6    = Dropout(drop_out)(ff_norm_6)
    ff_dense_7   = Dense(16, activation='relu')(ff_drop_6)
    ff_norm_7    = BatchNormalization()(ff_dense_7)
    ff_drop_7    = Dropout(drop_out)(ff_norm_7)
    ff_dense_8   = Dense(8, activation='relu')(ff_drop_7)
    ff_norm_8    = BatchNormalization()(ff_dense_8)
    ff_drop_8    = Dropout(drop_out)(ff_norm_8)
    ff_dense_9   = Dense(4, activation='relu')(ff_drop_8)

    rr_input     = Input(shape=input_shape_lstm)
    rr_lstm_1    = Bidirectional(LSTM(128, activation='tanh', return_sequences=False))(rr_input)
    rr_dense_1   = Dense(64, activation='relu')(rr_lstm_1)
    rr_dense_2   = Dense(32, activation='relu')(rr_dense_1)
    
    gr_input     = Input(shape=input_shape_lstm)
    gr_gru_1     = GRU(258)(gr_input)
    gr_gru_2     = GRU(128)(gr_input)
    gr_gru_3     = GRU(64)(gr_input)
    gr_gru_4     = GRU(32)(gr_input)

    merged       = concatenate([ff_dense_9, rr_dense_2, gr_gru_4])
    mg_dense_1   = Dense(16, activation='relu')(merged)
    
    mg_output    = Dense(num_targets, activation='linear')(mg_dense_1)

    model = Model(inputs=[ff_input, rr_input, gr_input], outputs=mg_output)
    
    return model

In [ ]:
do_separate = False
do_combined = True

In [ ]:
if do_separate:
    model_fflstm_f = create_model(num_targets=1)
    model_fflstm_f.compile(optimizer='adam', loss=msle)
    model_fflstm_f.fit(X_train, y_formation_train, epochs=10000, batch_size=batch_size,
                     validation_data=(X_val, y_formation_val), callbacks=callbacks)

In [ ]:
if do_separate:
    model_fflstm_b = create_model(num_targets=1)
    model_fflstm_b.compile(optimizer='adam', loss=msle)
    model_fflstm_b.fit(X_train, y_bandgap_train, epochs=10000, batch_size=batch_size,
                     validation_data=(X_val, y_bandgap_val), callbacks=callbacks)

In [ ]:
if do_separate:
    pred_fflstm_f = model_fflstm_f.predict(X_test)
    pred_fflstm_b = model_fflstm_b.predict(X_test)
    submission_fflstm_fb = pd.DataFrame(np.arange(1, size_test + 1), columns=['id'])
    submission_fflstm_fb['formation_energy_ev_natom'] = pred_fflstm_f
    submission_fflstm_fb['bandgap_energy_ev'] = pred_fflstm_b
    submission_fflstm_fb.to_csv('submission_fflstm_fb.csv', index=False)
    submission_fflstm_fb.shape

In [ ]:
model_fflstm_c = create_model(num_targets=2)
model_fflstm_c.compile(optimizer='adam', loss=msle)
model_fflstm_c.fit(X_train, y_train, epochs=10000, batch_size=batch_size,
                 validation_data=(X_val, y_val), callbacks=callbacks)

In [ ]:
pred_fflstm_c = model_fflstm_c.predict(X_test)
submission_fflstm_c = pd.DataFrame(np.arange(1, size_test + 1), columns=['id'])
pred_fflstm_cf = []
pred_fflstm_cb = []
for row in pred_fflstm_c:
    pred_fflstm_cf.append(row[0])
    pred_fflstm_cb.append(row[1])
submission_fflstm_c['formation_energy_ev_natom'] = pred_fflstm_cf
submission_fflstm_c['bandgap_energy_ev'] = pred_fflstm_cb
submission_fflstm_c.to_csv('submission_fflstm_c.csv', index=False)
submission_fflstm_c.shape

In [ ]:
plot_model(model_fflstm_c, 
           to_file='model_fflstm_c.png', 
           show_shapes=False,
           show_layer_names=False,
           show_layer_activations=False,
           show_trainable=False)

print(model_fflstm_c.summary())

Image(filename='model_fflstm_c.png')

# Feedforward Network Model

In [ ]:
#--- Feedforward Neural Network ---#

input_shape_ff =(len(df_comp_train.iloc[0]),)

def create_feedforward(num_targets=1, multimodal=False):
            
    model = Sequential()
    model.add(Dense(1024, activation='relu', input_shape=input_shape_ff))
    model.add(BatchNormalization())
    model.add(Dropout(drop_out))
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(drop_out))
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(drop_out))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(drop_out))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(drop_out))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(drop_out))
    model.add(Dense(16, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(drop_out))
    model.add(Dense(8, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(drop_out))
    model.add(Dense(4, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(drop_out))
    model.add(Dense(num_targets, activation='linear'))
    
    return model

In [ ]:
#--- Model predicting formation energy ---#=

model_ff_f = create_feedforward(num_targets=1)
model_ff_f.compile(optimizer='adam', loss=msle)
model_ff_f.fit(df_comp_train, y_formation_train, epochs=100, batch_size=batch_size, 
               validation_data=(df_comp_val, y_formation_val), callbacks=callbacks)

In [ ]:
#--- Model predicting bandgap energy ---#

model_ff_b = create_feedforward(num_targets=1)
model_ff_b.compile(optimizer='adam', loss=rmsle)
model_ff_b.fit(df_comp_train, y_bandgap_train, epochs=100, batch_size=batch_size, 
               validation_data=(df_comp_val, y_bandgap_val), callbacks=callbacks)

In [ ]:
#--- Save predictions ---#

pred_f_ff_fb = model_ff_f.predict(df_comp_test)
pred_b_ff_fb = model_ff_b.predict(df_comp_test)
submission_ff_fb = pd.DataFrame(np.arange(1, size_test + 1), columns=['id'])
submission_ff_fb['formation_energy_ev_natom'] = pred_f_ff_fb
submission_ff_fb['bandgap_energy_ev'] = pred_b_ff_fb
submission_ff_fb.to_csv('submission_ff_fb.csv', index=False)
submission_ff_fb.shape

In [ ]:
#--- Model predicting both targets ---#

model_ff_c = create_feedforward(num_targets=2)
model_ff_c.compile(optimizer='adam', loss=rmsle)
model_ff_c.fit(df_comp_train, y_train, epochs=100, batch_size=batch_size, 
               validation_data=(df_comp_val, y_val), callbacks=callbacks)

In [ ]:
#--- Save predictions ---#

pred_ff_c = model_ff_c.predict(df_comp_test)
submission_ff_c = pd.DataFrame(np.arange(1, size_test + 1), columns=['id'])
pred_f_ff_c = []
pred_b_ff_c = []
for row in pred:
    pred_f_ff_c.append(row[0])
    pred_b_ff_c.append(row[1])
submission_ff_c['formation_energy_ev_natom'] = pred_f_ff_c
submission_ff_c['bandgap_energy_ev'] = pred_b_ff_c
submission_ff_c.to_csv('submission_ff_c.csv', index=False)
submission_ff_c.shape

In [ ]:
plot_model(model_ff_c, 
           to_file='model_ff_c.png', 
           show_shapes=False,
           show_layer_names=False,
           show_layer_activations=False,
           show_trainable=False)

print(model_ff_c.summary())

Image(filename='model_ff_c.png')

# LSTM Model

In [ ]:
run_lstm_c = True

In [ ]:
input_shape_lstm = (80,1)

def create_lstm(num_targets=2):
            
    model = Sequential()
    model.add(Bidirectional(LSTM(32, activation='tanh', return_sequences=True, input_shape=input_shape_lstm)))
    model.add(Bidirectional(LSTM(32, activation='tanh', return_sequences=False)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_targets, activation='linear'))
    
    return model

In [ ]:
#--- LSTM Training ---#

if run_lstm_c:
    model_lstm_c = create_lstm(num_targets=2)
    model_lstm_c.compile(optimizer='adam', loss=msle)
    model_lstm_c.fit(arr_spectrum_train, y_train, epochs=10000, batch_size=batch_size, 
                     validation_data=(arr_spectrum_val, y_val), callbacks=callbacks)

In [ ]:
#--- Save predictions ---#

if run_lstm_c:
    pred_lstm_c = model_lstm_c.predict(arr_spectrum_test)
    submission_lstm_c = pd.DataFrame(np.arange(1, size_test + 1), columns=['id'])
    pred_f_lstm_c = []
    pred_b_lstm_c = []
    for row in pred_lstm_c:
        pred_f_lstm_c.append(row[0])
        pred_b_lstm_c.append(row[1])
    submission_lstm_c['formation_energy_ev_natom'] = pred_f_lstm_c
    submission_lstm_c['bandgap_energy_ev'] = pred_b_lstm_c
    submission_lstm_c.to_csv('submission_lstm_c.csv', index=False)
    submission_lstm_c.shape